In [385]:
import numpy as np
import math
from numpy.random import randint, choice

In [386]:
routing_table = np.array([
   # Car_1    Car_2    Car_3    Car_4    Car_5    Car_6    Car_7    Car_8    Car_9    Base_1   Base_2
    [-np.inf, 4,       4,       4,       3,       -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf],  # Car_1
    [4,       -np.inf, -np.inf, 5,       -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, 6,       -np.inf],  # Car_2
    [4,       -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf],  # Car_3
    [4,       5,       -np.inf, -np.inf, 4,       -np.inf, 1,       -np.inf, -np.inf, -np.inf, -np.inf],  # Car_4
    [3,       -np.inf, -np.inf, 4,       -np.inf, 5,       2,       7,       4,       -np.inf, -np.inf],  # Car_5
    [-np.inf, -np.inf, 3,       -np.inf, 5,       -np.inf, -np.inf, -np.inf, 5,       -np.inf, -np.inf],  # Car_6
    [-np.inf, -np.inf, -np.inf, 1,       2,       -np.inf, -np.inf, 3,       -np.inf, -np.inf, 6      ],  # Car_7
    [-np.inf, -np.inf, -np.inf, -np.inf, 7,       -np.inf, 3,       -np.inf, 4,       -np.inf, -np.inf],  # Car_8
    [-np.inf, -np.inf, -np.inf, -np.inf, 4,       5,       -np.inf, 4,       -np.inf, -np.inf, -np.inf],  # Car_9
    [-np.inf, 6,       -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf],  # Base_1
    [-np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, 6,       -np.inf, -np.inf, -np.inf, -np.inf],  # Base_2
])

In [387]:
routing_table_indices = {
    'car_1': 0, 'car_2': 1, 'car_3': 2, 'car_4': 3, 'car_5': 4, 'car_6': 5,\
    'car_7': 6, 'car_8': 7, 'car_9': 8, 'base_station_1': 9, 'base_station_2': 10
}

In [388]:
def init_population(origin, destination, pop_len):
    routes = []
    '''
    get indices that can be randomly used to create a population but 
    exclude the origin and destination since they will have a fixed position
    '''
    usable_route_indices = {car: idx for car, idx in routing_table_indices.items() if car != origin and car != destination}
    node_indices = list(usable_route_indices.values())
    # generate population of length defined as a parameter
    for i in range(pop_len):
        # length of path (chromosome) is not fixed so generate a random length of path for each iteration
        path_length = randint(1, len(node_indices))
        # choose indices to be used as genes
        chosen_indices = choice(np.array(node_indices), path_length, replace=False)
        # prepend and append origin and destination respectively to the generated path
        route = np.append(routing_table_indices[origin], chosen_indices)
        route = np.append(route, routing_table_indices[destination])
        # add route to list of routes (the population)
        routes.append(route)

    return routes

In [389]:
routes = init_population('car_9', 'base_station_2', 1000)
#print(routes)

In [390]:
def pop_fitness_combine(population, fitness_vals):
    pop_fitness_data = []
    for i in range(len(population)):
        pop_fitness_data.append((population[i], fitness_vals[i]))

    return pop_fitness_data

In [391]:
def bandwidth_calc(routes):
    route_distances = []

    for route in routes:
        total_distance = 0
        for i in range(len(route) - 1):   
            total_distance += routing_table[route[i]][route[i + 1]]
        route_distances.append(total_distance)
    pop_fitness_data = pop_fitness_combine(routes, route_distances)

    return pop_fitness_data

In [392]:
pop_performance = bandwidth_calc(routes)

In [393]:
def get_valid_paths(pop_performance):
    pop_fitness_data = []
    print("********************************************Performance**********************************")
    for individual in  pop_performance:
        if individual[1] > 0:
            path = ''
            dest_node_idx = len(individual[0]) - 1
            for idx, j in enumerate(individual[0]):
                for name, code in routing_table_indices.items():
                    if code == j:
                        path += name
                        if idx != dest_node_idx:
                            path += "->"
            print(f"Chromosome: {individual[0]}--------Actual route: {path}--------Score: {individual[1]}")
            pop_fitness_data.append(individual)
    print("*****************************************************************************************")

    return pop_fitness_data

In [394]:
pop_fitness_data = get_valid_paths(pop_performance)
pop_fitness_data

********************************************Performance**********************************
Chromosome: [ 8  7  6 10]--------Actual route: car_9->car_8->car_7->base_station_2--------Score: 13.0
Chromosome: [ 8  7  6 10]--------Actual route: car_9->car_8->car_7->base_station_2--------Score: 13.0
Chromosome: [ 8  7  6 10]--------Actual route: car_9->car_8->car_7->base_station_2--------Score: 13.0
*****************************************************************************************


[(array([ 8,  7,  6, 10]), 13.0),
 (array([ 8,  7,  6, 10]), 13.0),
 (array([ 8,  7,  6, 10]), 13.0)]

## The above are the valid paths from the population

In [396]:
def tournament_fitness_selection(pop_fitness_data, selection_limit=20):
    comparison_pairs = []
    selected_winners = []
    for i in range(len(pop_fitness_data)):
        possible_pair_indices = [j for j in range(len(pop_fitness_data)) if j != i]
        comparison_pairs.append((i, choice(possible_pair_indices)))
    for j in comparison_pairs:
        if pop_fitness_data[j[0]][1] < pop_fitness_data[j[1]][1]:
            selected_winners.append(pop_fitness_data[j[0]][0])
    if len(selected_winners) > selection_limit:
        selected_winners = selected_winners[:selection_limit]

    return selected_winners

In [397]:
selection_winners = tournament_fitness_selection(pop_fitness_data, selection_limit=20)

In [398]:
def crossover_parent_pairing(selection_winners):
    return [selection_winners[i:i+2] for i in range(0, len(selection_winners), 2)]

In [399]:
parents = crossover_parent_pairing(selection_winners)

In [400]:
def crossover(parents_list, type='single-point'):
    """
    crossover(parents_list, type='single-point')
    function that performs a crossover of 2 parents in the population
    and returns 2 children
    The type of crossover is dependent on the argument passed in as an
    argument. Crossover is done by breaking down the chromosome at randomly
    selected points and concatenating with the other parents genes from that
    position. This is done without changing the genes of the parents since new
    children chromosomes are created.
    parents_list: a list of 2 parents that need a crossover done on them
    type(optional): the type of crossover desired to be done. The default type 
    is single point
    returns: a list of 2 children that result from the crossover
    """
    children_list = []

    if type == 'single-point':
        # if the type is set to single-point, perform a single-point crossover
        crossover_point = randint(1, len(parents_list[0])-1)
        '''
        slice the list from beginning to randomly selected index,
        append slice of second list from that point to the end to the first.
        perform the inverse on the second parent
        '''
        children_list.append([*parents_list[0][:crossover_point],\
                                  *parents_list[1][crossover_point:]])
        children_list.append([*parents_list[1][:crossover_point],\
                                  *parents_list[0][crossover_point:]])
    elif type == 'two-point':
        # if the type is set to two-point, perform a two-point crossover
        crossover_point = randint(low=1, high=len(parents_list[0])-1, size=2)
        '''
        slice the list from beginning to the first randomly selected index, insert 
        part of the second part from that index up until the second randomly selected 
        index and add the last part of the forst index from that point to the end
        perform the inverse on the second parent
        '''
        children_list.append([*parents_list[0][:crossover_point[0]],\
                                  *parents_list[1][crossover_point[0]:crossover_point[1]],\
                                  *parents_list[0][crossover_point[1]:]])
        children_list.append([*parents_list[1][:crossover_point[0]],\
                                  *parents_list[0][crossover_point[0]:crossover_point[1]],\
                                  *parents_list[1][crossover_point[1]:]])
    elif type == 'uniform':
        # if the type is set to uniform, perform uniform crossover
        child1 = []
        child2 = []
        # randomly and uniformly swap bits between the two parents
        for i in range(len(parents_list[0])):
            crossover_point = randint(0, 2)
            if crossover_point == 1:
                child1.append(parents_list[1][i])
                child2.append(parents_list[0][i])
            else:
                child1.append(parents_list[0][i])
                child2.append(parents_list[1][i])
        children_list.append(child1)
        children_list.append(child2)
                
    return np.array(children_list)

In [401]:
children = []
for i in parents:
    if len(i) == 2:
        crossover_result = crossover(i)
        for j in crossover_result:
            children.append(j)
parent_children = np.array(selection_winners + children)
print(parent_children)

[]


In [402]:
def mutate(bin_list, mutation_rate=0.1):
    """
    mutate(bin_list, mutation_rate=0.1)
    function that performs mutation of random individuals in
    a binary population. The number of individuals on which mutation
    will take place and the number of bits in an individual affected by
    the mutation are determined by the mutation rate and mutation ratio
    respectively
    bin_list: the binary population on in which mutation will is performed
    mutation_rate(optional): The percentage of the populations being mutated
    mutation_ratio(optional): The percentage of bits in the population being mutated
    returns: the population that has been mutated based in the parameters provided
    """
    '''
    randomly obtain the indices of the individuals being mutated. 
    The number of indices will approximately be based on the percentage 
    provided as the mutation_rate parameter
    '''
    mutation_chromosome_indices = choice(np.array(range(len(bin_list))),\
                                         math.ceil(len(bin_list) * mutation_rate),\
                                         replace=False)

    print("Mutation Chromosome indices:"+str(mutation_chromosome_indices))
    # loop through each individual in the population
    for i in mutation_chromosome_indices:
        '''
        randomly obtain the indices of the bits being mutated. 
        The number of indices will approximately be based on the percentage 
        provided as the mutation_ratio parameter
        '''
        mutation_gene_indices = choice(np.array(range(len(bin_list[i]))),\
                                         2,\
                                         replace=False)
        # swap the genes
        temp = bin_list[i][mutation_gene_indices[0]]
        bin_list[i][mutation_gene_indices[0]] = bin_list[i][mutation_gene_indices[1]]
        bin_list[i][mutation_gene_indices[1]] = temp
        
    return np.array(bin_list)

In [403]:
print("Population before mutation:\n"+str(parent_children))
print("------------------------------------------")
print("Population after mutation:\n"+str(mutate(parent_children)))

Population before mutation:
[]
------------------------------------------
Mutation Chromosome indices:[]
Population after mutation:
[]


In [404]:
def discrete_genetic_algorithm(init_bin_population, generations=10):
    population = init_bin_population
    for i in range(generations):
        print(f"**********************************Generation {i + 1}**********************************")
        pop_performance = bandwidth_calc(routes)
        pop_fitness_data = get_valid_paths(pop_performance)
        selection_winners = tournament_fitness_selection(pop_fitness_data)
        parents = crossover_parent_pairing(selection_winners)
        children = []
        for i in parents:
            if len(i) == 2:
                crossover_result = crossover(i)
                for j in crossover_result:
                    children.append(j)
        parent_children = np.array(selection_winners + children)
        population = mutate(parent_children)

# Implementation

In [406]:
routes = init_population('car_9', 'base_station_2', 10000)

In [407]:
discrete_genetic_algorithm(routes)

**********************************Generation 1**********************************
********************************************Performance**********************************
Chromosome: [ 8  4  7  6 10]--------Actual route: car_9->car_5->car_8->car_7->base_station_2--------Score: 20.0
Chromosome: [ 8  7  6 10]--------Actual route: car_9->car_8->car_7->base_station_2--------Score: 13.0
Chromosome: [ 8  4  6 10]--------Actual route: car_9->car_5->car_7->base_station_2--------Score: 12.0
Chromosome: [ 8  7  6 10]--------Actual route: car_9->car_8->car_7->base_station_2--------Score: 13.0
Chromosome: [ 8  7  6 10]--------Actual route: car_9->car_8->car_7->base_station_2--------Score: 13.0
Chromosome: [ 8  7  6 10]--------Actual route: car_9->car_8->car_7->base_station_2--------Score: 13.0
Chromosome: [ 8  7  4  6 10]--------Actual route: car_9->car_8->car_5->car_7->base_station_2--------Score: 19.0
Chromosome: [ 8  7  6 10]--------Actual route: car_9->car_8->car_7->base_station_2--------Score

/var/folders/3m/7yx097tn2wz5t7h3fbj4dj_00000gn/T/ipykernel_21340/3105440398.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(children_list)
/var/folders/3m/7yx097tn2wz5t7h3fbj4dj_00000gn/T/ipykernel_21340/4195352018.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  parent_children = np.array(selection_winners + children)
